## LDA IMPLEMENTATION 

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('test_data.csv')
final_df=pd.DataFrame()
final_df['Feature']=['temperature',
        'tyre1_press',
        'tyre2_press',
        'tyre3_press',
        'tyre4_press',
        'tyre5_press',
        'tyre6_press',
        'avg_speed',
        'time',
        'fuel',]
feature_dict = {i:label for i,label in zip(
                range(21),
                  ('load_type',
                   'truck_type',
                   'route_type',
                   'rainfall',
                   'temperature',
                   'temp_type',
                   'tyre1_press',
                   'tyre1_press_stat',
                   'tyre2_press',
                   'tyre2_press_stat',
                   'tyre3_press',
                   'tyre3_press_stat',
                   'tyre4_press',
                   'tyre4_press_stat',
                   'tyre5_press',
                   'tyre5_press_stat',
                   'tyre6_press',
                   'tyre6_press_stat',
                   'avg_speed',
                   'time',
                   'fuel',
                   ))}

df.columns = [ l for i,l in sorted(feature_dict.items())] + [ 'tyre1_condition',
                                                               'tyre2_condition',
                                                               'tyre3_condition',
                                                               'tyre4_condition',
                                                               'tyre5_condition',
                                                               'tyre6_condition',
                                                               'efficiency',      ]
df.dropna(how="all", inplace=True) # to drop the empty line at file-end

df.head()

,load_type,truck_type,route_type,rainfall,temperature,temp_type,tyre1_press,tyre1_press_stat,tyre2_press,tyre2_press_stat,...,avg_speed,time,fuel,tyre1_condition,tyre2_condition,tyre3_condition,tyre4_condition,tyre5_condition,tyre6_condition,efficiency
0,Moderate,FTL,Bumpy,N,28,Normal,40,Correct,64,Correct,...,53,9,33,tyre1_not flat,tyre2_not flat,tyre3_not flat,tyre4_not flat,tyre5_flat,tyre6_not flat,0.604936
1,Heavy Load,FTL,Hilly,Y,44,High,67,High,40,Correct,...,51,13,35,tyre1_flat,tyre2_not flat,tyre3_not flat,tyre4_not flat,tyre5_not flat,tyre6_not flat,0.215965
2,Full Load,FTL,Hilly,N,32,Normal,48,Correct,68,High,...,51,6,19,tyre1_not flat,tyre2_flat,tyre3_not flat,tyre4_not flat,tyre5_not flat,tyre6_not flat,0.813573
3,Heavy Load,FTL,Plain,N,26,Normal,52,Correct,60,Correct,...,48,6,80,tyre1_not flat,tyre2_not flat,tyre3_not flat,tyre4_not flat,tyre5_not flat,tyre6_not flat,0.537890
4,Moderate,LCV,Hilly,N,25,Normal,61,Correct,51,Correct,...,51,5,26,tyre1_not flat,tyre2_not flat,tyre3_not flat,tyre4_not flat,tyre5_not flat,tyre6_not flat,0.061304


In [2]:
from sklearn.preprocessing import LabelEncoder

X = df[['temperature',
        'tyre1_press',
        'tyre2_press',
        'tyre3_press',
        'tyre4_press',
        'tyre5_press',
        'tyre6_press',
        'avg_speed',
        'time',
        'fuel',]].values

y = df['tyre1_condition'].values
from sklearn import preprocessing

#min_max_scaler = preprocessing.MinMaxScaler()
#X_train_minmax = min_max_scaler.fit_transform(X[0:7])
#X[0:7]=X_train_minmax

from sklearn.preprocessing import StandardScaler
X[0:7] = StandardScaler().fit_transform(X[0:7])

#from sklearn.preprocessing import RobustScaler
#transformer = RobustScaler().fit(X[0:7])
#transformer.transform(X[0:7])

#from sklearn.preprocessing import MaxAbsScaler
#transformer = MaxAbsScaler().fit(X[0:7])
#transformer.transform(X[0:7])

##X_normalized = preprocessing.normalize(X, norm='l2')

#normalizer = preprocessing.Normalizer().fit(X[0:7])
#normalizer.transform(X[0:7])

enc = LabelEncoder()
label_encoder = enc.fit(y)
y = label_encoder.transform(y) + 1

label_dict = {1: 'tyre_flat', 2: 'tyre_not flat'}

np.set_printoptions(precision=4)

mean_vectors = []
for cl in range(1,3):
    mean_vectors.append(np.mean(X[y==cl], axis=0))
    print('Mean Vector class %s: %s\n' %(cl, mean_vectors[cl-1]))


Mean Vector class 1: [ 28.5     46.9474  46.8947  51.0789  37.      47.5263  49.5789  48.8421
  12.4737  43.8421]

Mean Vector class 2: [ 28.2972  48.033   48.7358  49.8255  37.3774  47.5708  49.1132  48.4198
   8.6226  50.9764]



C:\Users\Pooja\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [3]:
np.set_printoptions(precision=4)   
S_W = np.zeros((10,10))
for cl,mv in zip(range(1,10), mean_vectors):
    class_sc_mat = np.zeros((10,10))                  # scatter matrix for every class
    for row in X[y == cl]:
        row, mv = row.reshape(10,1), mv.reshape(10,1) # make column vectors
        class_sc_mat += (row-mv).dot((row-mv).T)
    S_W += class_sc_mat  
                           # sum class scatter matrices
print('within-class Scatter Matrix:\n', S_W)
np.set_printoptions(precision=4)
overall_mean = np.mean(X, axis=0)


within-class Scatter Matrix:
 [[  10895.7783   16377.9198    9913.6415    8205.4953    7678.2264
     8401.0425    9126.8679    9985.5519    1818.7736   10673.4858]
 [  16377.9198   44894.6636   18561.6385   12583.3796   12081.3585
    15704.0573   17712.3654   17488.7455    3283.5889   16795.8493]
 [   9913.6415   18561.6385   40928.7865   19375.5422   12352.1321
    18255.0675   16524.6554   17623.8779    3245.7627   22977.0477]
 [   8205.4953   12583.3796   19375.5422   41673.3056   13063.9623
    17049.539    15658.4518   18122.0067    2539.6167   13637.601 ]
 [   7678.2264   12081.3585   12352.1321   13063.9623   13561.8113
    12917.3396   13735.9434   13363.4151    2409.1887   13276.8868]
 [   8401.0425   15704.0573   18255.0675   17049.539    12917.3396
    33793.4124   14606.7229   16396.3607    3558.1867   15177.0117]
 [   9126.8679   17712.3654   16524.6554   15658.4518   13735.9434
    14606.7229   36004.5462   17912.3982    3364.6356   15423.0397]
 [   9985.5519   17488.74

In [4]:
S_B = np.zeros((10,10))
for i,mean_vec in enumerate(mean_vectors):  
    n = X[y==i+1,:].shape[0]
    mean_vec = mean_vec.reshape(10,1) 
    overall_mean = overall_mean.reshape(10,1) 
    S_B += n * (mean_vec - overall_mean).dot((mean_vec - overall_mean).T)

print('between-class Scatter Matrix:\n', S_B)



between-class Scatter Matrix:
 [[  1.3257e+00  -7.0958e+00  -1.2034e+01   8.1927e+00  -2.4664e+00
   -2.9045e-01   3.0441e+00   2.7601e+00   2.5170e+01  -4.6630e+01]
 [ -7.0958e+00   3.7980e+01   6.4409e+01  -4.3852e+01   1.3202e+01
    1.5547e+00  -1.6293e+01  -1.4774e+01  -1.3472e+02   2.4959e+02]
 [ -1.2034e+01   6.4409e+01   1.0923e+02  -7.4366e+01   2.2388e+01
    2.6365e+00  -2.7631e+01  -2.5054e+01  -2.2847e+02   4.2326e+02]
 [  8.1927e+00  -4.3852e+01  -7.4366e+01   5.0630e+01  -1.5242e+01
   -1.7950e+00   1.8812e+01   1.7057e+01   1.5555e+02  -2.8817e+02]
 [ -2.4664e+00   1.3202e+01   2.2388e+01  -1.5242e+01   4.5887e+00
    5.4038e-01  -5.6634e+00  -5.1351e+00  -4.6829e+01   8.6753e+01]
 [ -2.9045e-01   1.5547e+00   2.6365e+00  -1.7950e+00   5.4038e-01
    6.3637e-02  -6.6694e-01  -6.0472e-01  -5.5147e+00   1.0216e+01]
 [  3.0441e+00  -1.6293e+01  -2.7631e+01   1.8812e+01  -5.6634e+00
   -6.6694e-01   6.9898e+00   6.3378e+00   5.7796e+01  -1.0707e+02]
 [  2.7601e+00  -1.4774e

In [5]:
np.set_printoptions(precision=4)
eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))
np.set_printoptions(precision=4)
for i in range(len(eig_vals)):
    eigvec_sc = eig_vecs[:,i].reshape(10,1)  
    np.set_printoptions(precision=4)
    print('\nEigenvector {}: \n{}'.format(i+1, eigvec_sc.real))
    np.set_printoptions(precision=4)
    print('Eigenvalue {:}: {:.2e}'.format(i+1, eig_vals[i].real))
    
for i in range(len(eig_vals)):
    eigv = eig_vecs[:,i].reshape(10,1)
    np.testing.assert_array_almost_equal(np.linalg.inv(S_W).dot(S_B).dot(eigv),
                                         eig_vals[i] * eigv,
                                         decimal=6, err_msg='', verbose=True)
print('Eigen Vectors ok')


eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

eig_pairs = sorted(eig_pairs, key=lambda k: k[0], reverse=True)

#print('Eigenvalues in decreasing order:\n')
#for i in eig_pairs:
#    print(i[0])
scores=[]
print('\nConverting to percentage:\n')
eig_vals=abs(eig_vals)
eigv_sum = sum(eig_vals)

for i,j in enumerate(eig_pairs):
    print('eigenvalue {0:}: {1:}'.format(j[0], (j[0]/eigv_sum).real))
    scores.append((j[0]/eigv_sum).real)

final_df['Tyre 1']=scores
    
W = np.hstack((eig_pairs[0][1].reshape(10,1)))
print('Matrix W:\n', W.real)

X_lda = X.dot(W).reshape(-1,1)
assert X_lda.shape == (250,1), "The matrix is not 250x1 dimensional."
eig_vals


Eigenvector 1: 
[[ -9.9919e-01]
 [ -2.2815e-03]
 [ -5.5643e-04]
 [  1.1892e-03]
 [ -3.2868e-03]
 [ -1.4500e-03]
 [ -2.3005e-04]
 [  1.9585e-03]
 [  3.9488e-02]
 [ -6.1084e-03]]
Eigenvalue 1: 0.00e+00

Eigenvector 2: 
[[ 0.1162]
 [-0.0563]
 [-0.0137]
 [ 0.0293]
 [-0.0811]
 [-0.0358]
 [-0.0057]
 [ 0.0483]
 [ 0.9743]
 [-0.1507]]
Eigenvalue 2: 2.75e-01

Eigenvector 3: 
[[ 0.3568]
 [ 0.6828]
 [ 0.0262]
 [ 0.0141]
 [ 0.0636]
 [-0.0392]
 [-0.0031]
 [-0.0444]
 [ 0.1463]
 [-0.025 ]]
Eigenvalue 3: 5.23e-18

Eigenvector 4: 
[[ 0.3568]
 [ 0.6828]
 [ 0.0262]
 [ 0.0141]
 [ 0.0636]
 [-0.0392]
 [-0.0031]
 [-0.0444]
 [ 0.1463]
 [-0.025 ]]
Eigenvalue 4: 5.23e-18

Eigenvector 5: 
[[ 0.4916]
 [-0.0231]
 [-0.0363]
 [-0.2216]
 [-0.5222]
 [ 0.0111]
 [ 0.0777]
 [ 0.6505]
 [-0.0698]
 [ 0.0214]]
Eigenvalue 5: -1.27e-18

Eigenvector 6: 
[[ 0.5131]
 [ 0.1182]
 [ 0.0334]
 [-0.2718]
 [ 0.2748]
 [-0.1342]
 [-0.0182]
 [-0.3547]
 [ 0.0455]
 [-0.0711]]
Eigenvalue 6: 4.10e-19

Eigenvector 7: 
[[ 0.5131]
 [ 0.1182]
 [ 0

array([  0.0000e+00,   2.7493e-01,   1.8994e-17,   1.8994e-17,
         1.2717e-18,   6.2165e-19,   6.2165e-19,   2.3905e-19,
         3.9147e-20,   3.9147e-20])

In [6]:
################### TYRE 2 #########################################
print ('################### TYRE 2 ###################')
y = df['tyre2_condition'].values

enc = LabelEncoder()
label_encoder = enc.fit(y)
y = label_encoder.transform(y) + 1

label_dict = {1: 'tyre_flat', 2: 'tyre_not flat'}

np.set_printoptions(precision=4)

mean_vectors = []
for cl in range(1,3):
    mean_vectors.append(np.mean(X[y==cl], axis=0))
#    print('Mean Vector class %s: %s\n' %(cl, mean_vectors[cl-1]))
    
np.set_printoptions(precision=4)   
S_W = np.zeros((10,10))
for cl,mv in zip(range(1,10), mean_vectors):
    class_sc_mat = np.zeros((10,10))                  # scatter matrix for every class
    for row in X[y == cl]:
        row, mv = row.reshape(10,1), mv.reshape(10,1) # make column vectors
        class_sc_mat += (row-mv).dot((row-mv).T)
    S_W += class_sc_mat  
                           # sum class scatter matrices
#print('within-class Scatter Matrix:\n', S_W)
np.set_printoptions(precision=4)
overall_mean = np.mean(X, axis=0)

S_B = np.zeros((10,10))
for i,mean_vec in enumerate(mean_vectors):  
    n = X[y==i+1,:].shape[0]
    mean_vec = mean_vec.reshape(10,1) 
    overall_mean = overall_mean.reshape(10,1) 
    S_B += n * (mean_vec - overall_mean).dot((mean_vec - overall_mean).T)

#print('between-class Scatter Matrix:\n', S_B)

np.set_printoptions(precision=4)
eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))

#for i in range(len(eig_vals)):
#    eigvec_sc = eig_vecs[:,i].reshape(10,1)   
#    print('\nEigenvector {}: \n{}'.format(i+1, eigvec_sc.real))
#    print('Eigenvalue {:}: {:.2e}'.format(i+1, eig_vals[i].real))
#    
for i in range(len(eig_vals)):
    eigv = eig_vecs[:,i].reshape(10,1)
    np.testing.assert_array_almost_equal(np.linalg.inv(S_W).dot(S_B).dot(eigv),
                                         eig_vals[i] * eigv,
                                         decimal=6, err_msg='', verbose=True)
#print('Eigen Vectors ok')

eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

eig_pairs = sorted(eig_pairs, key=lambda k: k[0], reverse=True)

#print('Eigenvalues in decreasing order:\n')
#for i in eig_pairs:
#    print(i[0])
scores=[]
print('\nConverting to percentage:\n')
eigv_sum = sum(eig_vals)
for i,j in enumerate(eig_pairs):
    print('eigenvalue {0:}: {1:.2%}'.format(i+1, (j[0]/eigv_sum).real))
    scores.append((j[0]/eigv_sum).real)

final_df['Tyre 2']=scores
    
W = np.hstack((eig_pairs[0][1].reshape(10,1)))
print('Matrix W:\n', W.real)

X_lda = X.dot(W).reshape(-1,1)
assert X_lda.shape == (250,1), "The matrix is not 250x1 dimensional."



################### TYRE 2 ###################

Converting to percentage:

eigenvalue 1: 100.00%
eigenvalue 2: 0.00%
eigenvalue 3: 0.00%
eigenvalue 4: 0.00%
eigenvalue 5: 0.00%
eigenvalue 6: 0.00%
eigenvalue 7: 0.00%
eigenvalue 8: 0.00%
eigenvalue 9: 0.00%
eigenvalue 10: 0.00%
Matrix W:
 [ 0.3704 -0.1015 -0.0415  0.2692  0.2447 -0.1711  0.1548 -0.5214  0.6233
 -0.0699]


In [7]:
################### TYRE 3 #########################################
print ('################### TYRE 3 ###################')
y = df['tyre3_condition'].values

enc = LabelEncoder()
label_encoder = enc.fit(y)
y = label_encoder.transform(y) + 1

label_dict = {1: 'tyre_flat', 2: 'tyre_not flat'}

np.set_printoptions(precision=4)

mean_vectors = []
for cl in range(1,3):
    mean_vectors.append(np.mean(X[y==cl], axis=0))
#    print('Mean Vector class %s: %s\n' %(cl, mean_vectors[cl-1]))
    
np.set_printoptions(precision=4)   
S_W = np.zeros((10,10))
for cl,mv in zip(range(1,10), mean_vectors):
    class_sc_mat = np.zeros((10,10))                  # scatter matrix for every class
    for row in X[y == cl]:
        row, mv = row.reshape(10,1), mv.reshape(10,1) # make column vectors
        class_sc_mat += (row-mv).dot((row-mv).T)
    S_W += class_sc_mat  
                           # sum class scatter matrices
#print('within-class Scatter Matrix:\n', S_W)
np.set_printoptions(precision=4)
overall_mean = np.mean(X, axis=0)

S_B = np.zeros((10,10))
for i,mean_vec in enumerate(mean_vectors):  
    n = X[y==i+1,:].shape[0]
    mean_vec = mean_vec.reshape(10,1) 
    overall_mean = overall_mean.reshape(10,1) 
    S_B += n * (mean_vec - overall_mean).dot((mean_vec - overall_mean).T)

#print('between-class Scatter Matrix:\n', S_B)

np.set_printoptions(precision=4)
eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))

#for i in range(len(eig_vals)):
#    eigvec_sc = eig_vecs[:,i].reshape(10,1)   
#    print('\nEigenvector {}: \n{}'.format(i+1, eigvec_sc.real))
#    print('Eigenvalue {:}: {:.2e}'.format(i+1, eig_vals[i].real))
#    
for i in range(len(eig_vals)):
    eigv = eig_vecs[:,i].reshape(10,1)
    np.testing.assert_array_almost_equal(np.linalg.inv(S_W).dot(S_B).dot(eigv),
                                         eig_vals[i] * eigv,
                                         decimal=6, err_msg='', verbose=True)
#print('Eigen Vectors ok')

eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

eig_pairs = sorted(eig_pairs, key=lambda k: k[0], reverse=True)

#print('Eigenvalues in decreasing order:\n')
#for i in eig_pairs:
#    print(i[0])
scores=[]
print('\nConverting to percentage:\n')
eigv_sum = sum(eig_vals)
for i,j in enumerate(eig_pairs):
    print('eigenvalue {0:}: {1:.2%}'.format(i+1, (j[0]/eigv_sum).real))
    scores.append((j[0]/eigv_sum).real)

final_df['Tyre 3']=scores
    
W = np.hstack((eig_pairs[0][1].reshape(10,1)))
print('Matrix W:\n', W.real)

X_lda = X.dot(W).reshape(-1,1)
assert X_lda.shape == (250,1), "The matrix is not 250x1 dimensional."



################### TYRE 3 ###################

Converting to percentage:

eigenvalue 1: 100.00%
eigenvalue 2: 0.00%
eigenvalue 3: 0.00%
eigenvalue 4: 0.00%
eigenvalue 5: 0.00%
eigenvalue 6: 0.00%
eigenvalue 7: 0.00%
eigenvalue 8: 0.00%
eigenvalue 9: 0.00%
eigenvalue 10: 0.00%
Matrix W:
 [-0.4136  0.1793  0.1492  0.3476  0.6698 -0.2516 -0.2111 -0.1354 -0.242
 -0.1426]


In [8]:
################### TYRE 4 #########################################
print ('################### TYRE 4 ###################')
y = df['tyre4_condition'].values

enc = LabelEncoder()
label_encoder = enc.fit(y)
y = label_encoder.transform(y) + 1

label_dict = {1: 'tyre_flat', 2: 'tyre_not flat'}

np.set_printoptions(precision=4)

mean_vectors = []
for cl in range(1,3):
    mean_vectors.append(np.mean(X[y==cl], axis=0))
#    print('Mean Vector class %s: %s\n' %(cl, mean_vectors[cl-1]))
    
np.set_printoptions(precision=4)   
S_W = np.zeros((10,10))
for cl,mv in zip(range(1,10), mean_vectors):
    class_sc_mat = np.zeros((10,10))                  # scatter matrix for every class
    for row in X[y == cl]:
        row, mv = row.reshape(10,1), mv.reshape(10,1) # make column vectors
        class_sc_mat += (row-mv).dot((row-mv).T)
    S_W += class_sc_mat  
                           # sum class scatter matrices
#print('within-class Scatter Matrix:\n', S_W)
np.set_printoptions(precision=4)
overall_mean = np.mean(X, axis=0)

S_B = np.zeros((10,10))
for i,mean_vec in enumerate(mean_vectors):  
    n = X[y==i+1,:].shape[0]
    mean_vec = mean_vec.reshape(10,1) 
    overall_mean = overall_mean.reshape(10,1) 
    S_B += n * (mean_vec - overall_mean).dot((mean_vec - overall_mean).T)

#print('between-class Scatter Matrix:\n', S_B)

np.set_printoptions(precision=4)
eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))

#for i in range(len(eig_vals)):
#    eigvec_sc = eig_vecs[:,i].reshape(10,1)   
#    print('\nEigenvector {}: \n{}'.format(i+1, eigvec_sc.real))
#    print('Eigenvalue {:}: {:.2e}'.format(i+1, eig_vals[i].real))
#    
for i in range(len(eig_vals)):
    eigv = eig_vecs[:,i].reshape(10,1)
    np.testing.assert_array_almost_equal(np.linalg.inv(S_W).dot(S_B).dot(eigv),
                                         eig_vals[i] * eigv,
                                         decimal=6, err_msg='', verbose=True)
#print('Eigen Vectors ok')

eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

eig_pairs = sorted(eig_pairs, key=lambda k: k[0], reverse=True)

#print('Eigenvalues in decreasing order:\n')
#for i in eig_pairs:
#    print(i[0])
scores=[]
print('\nConverting to percentage:\n')
eigv_sum = sum(eig_vals)
for i,j in enumerate(eig_pairs):
    print('eigenvalue {0:}: {1:.2%}'.format(i+1, (j[0]/eigv_sum).real))
    scores.append((j[0]/eigv_sum).real)

final_df['Tyre 4']=scores
    
W = np.hstack((eig_pairs[0][1].reshape(10,1)))
print('Matrix W:\n', W.real)

X_lda = X.dot(W).reshape(-1,1)
assert X_lda.shape == (250,1), "The matrix is not 250x1 dimensional."



################### TYRE 4 ###################

Converting to percentage:

eigenvalue 1: 100.00%
eigenvalue 2: 0.00%
eigenvalue 3: 0.00%
eigenvalue 4: 0.00%
eigenvalue 5: 0.00%
eigenvalue 6: 0.00%
eigenvalue 7: 0.00%
eigenvalue 8: 0.00%
eigenvalue 9: 0.00%
eigenvalue 10: 0.00%
Matrix W:
 [ 0.1339 -0.0139 -0.0094  0.0398 -0.9142  0.0893  0.0896  0.3486  0.0825
  0.0013]


In [9]:
################### TYRE 5 #########################################
print ('################### TYRE 5 ###################')
y = df['tyre5_condition'].values

enc = LabelEncoder()
label_encoder = enc.fit(y)
y = label_encoder.transform(y) + 1

label_dict = {1: 'tyre_flat', 2: 'tyre_not flat'}

np.set_printoptions(precision=4)

mean_vectors = []
for cl in range(1,3):
    mean_vectors.append(np.mean(X[y==cl], axis=0))
#    print('Mean Vector class %s: %s\n' %(cl, mean_vectors[cl-1]))
    
np.set_printoptions(precision=4)   
S_W = np.zeros((10,10))
for cl,mv in zip(range(1,10), mean_vectors):
    class_sc_mat = np.zeros((10,10))                  # scatter matrix for every class
    for row in X[y == cl]:
        row, mv = row.reshape(10,1), mv.reshape(10,1) # make column vectors
        class_sc_mat += (row-mv).dot((row-mv).T)
    S_W += class_sc_mat  
                           # sum class scatter matrices
#print('within-class Scatter Matrix:\n', S_W)
np.set_printoptions(precision=4)
overall_mean = np.mean(X, axis=0)

S_B = np.zeros((10,10))
for i,mean_vec in enumerate(mean_vectors):  
    n = X[y==i+1,:].shape[0]
    mean_vec = mean_vec.reshape(10,1) 
    overall_mean = overall_mean.reshape(10,1) 
    S_B += n * (mean_vec - overall_mean).dot((mean_vec - overall_mean).T)

#print('between-class Scatter Matrix:\n', S_B)

np.set_printoptions(precision=4)
eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))

#for i in range(len(eig_vals)):
#    eigvec_sc = eig_vecs[:,i].reshape(10,1)   
#    print('\nEigenvector {}: \n{}'.format(i+1, eigvec_sc.real))
#    print('Eigenvalue {:}: {:.2e}'.format(i+1, eig_vals[i].real))
#    
for i in range(len(eig_vals)):
    eigv = eig_vecs[:,i].reshape(10,1)
    np.testing.assert_array_almost_equal(np.linalg.inv(S_W).dot(S_B).dot(eigv),
                                         eig_vals[i] * eigv,
                                         decimal=6, err_msg='', verbose=True)
#print('Eigen Vectors ok')

eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

eig_pairs = sorted(eig_pairs, key=lambda k: k[0], reverse=True)

#print('Eigenvalues in decreasing order:\n')
#for i in eig_pairs:
#    print(i[0])
scores=[]
print('\nConverting to percentage:\n')
eigv_sum = sum(eig_vals)
for i,j in enumerate(eig_pairs):
    print('eigenvalue {0:}: {1:.2%}'.format(i+1, (j[0]/eigv_sum).real))
    scores.append((j[0]/eigv_sum).real)

final_df['Tyre 5']=scores
    
W = np.hstack((eig_pairs[0][1].reshape(10,1)))
print('Matrix W:\n', W.real)

X_lda = X.dot(W).reshape(-1,1)
assert X_lda.shape == (250,1), "The matrix is not 250x1 dimensional."



################### TYRE 5 ###################

Converting to percentage:

eigenvalue 1: 100.00%
eigenvalue 2: 0.00%
eigenvalue 3: 0.00%
eigenvalue 4: 0.00%
eigenvalue 5: 0.00%
eigenvalue 6: 0.00%
eigenvalue 7: 0.00%
eigenvalue 8: 0.00%
eigenvalue 9: 0.00%
eigenvalue 10: 0.00%
Matrix W:
 [ 0.0206 -0.3275  0.0141 -0.4063 -0.0271 -0.6335 -0.082   0.4909  0.2621
 -0.093 ]


In [10]:
################### TYRE 6 #########################################
print ('################### TYRE 6 ###################')
y = df['tyre6_condition'].values

enc = LabelEncoder()
label_encoder = enc.fit(y)
y = label_encoder.transform(y) + 1

label_dict = {1: 'tyre_flat', 2: 'tyre_not flat'}

np.set_printoptions(precision=4)

mean_vectors = []
for cl in range(1,3):
    mean_vectors.append(np.mean(X[y==cl], axis=0))
#    print('Mean Vector class %s: %s\n' %(cl, mean_vectors[cl-1]))
    
np.set_printoptions(precision=4)   
S_W = np.zeros((10,10))
for cl,mv in zip(range(1,10), mean_vectors):
    class_sc_mat = np.zeros((10,10))                  # scatter matrix for every class
    for row in X[y == cl]:
        row, mv = row.reshape(10,1), mv.reshape(10,1) # make column vectors
        class_sc_mat += (row-mv).dot((row-mv).T)
    S_W += class_sc_mat  
                           # sum class scatter matrices
#print('within-class Scatter Matrix:\n', S_W)
np.set_printoptions(precision=4)
overall_mean = np.mean(X, axis=0)

S_B = np.zeros((10,10))
for i,mean_vec in enumerate(mean_vectors):  
    n = X[y==i+1,:].shape[0]
    mean_vec = mean_vec.reshape(10,1) 
    overall_mean = overall_mean.reshape(10,1) 
    S_B += n * (mean_vec - overall_mean).dot((mean_vec - overall_mean).T)

#print('between-class Scatter Matrix:\n', S_B)

np.set_printoptions(precision=4)
eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))

#for i in range(len(eig_vals)):
#    eigvec_sc = eig_vecs[:,i].reshape(10,1)   
#    print('\nEigenvector {}: \n{}'.format(i+1, eigvec_sc.real))
#    print('Eigenvalue {:}: {:.2e}'.format(i+1, eig_vals[i].real))
#    
for i in range(len(eig_vals)):
    eigv = eig_vecs[:,i].reshape(10,1)
    np.testing.assert_array_almost_equal(np.linalg.inv(S_W).dot(S_B).dot(eigv),
                                         eig_vals[i] * eigv,
                                         decimal=6, err_msg='', verbose=True)
#print('Eigen Vectors ok')

eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

eig_pairs = sorted(eig_pairs, key=lambda k: k[0], reverse=True)

#print('Eigenvalues in decreasing order:\n')
#for i in eig_pairs:
#    print(i[0])
scores=[]
print('\nConverting to percentage:\n')
eigv_sum = sum(eig_vals)
for i,j in enumerate(eig_pairs):
    print('eigenvalue {0:}: {1:.2%}'.format(i+1, (j[0]/eigv_sum).real))
    scores.append((j[0]/eigv_sum).real)

final_df['Tyre 6']=scores
    
W = np.hstack((eig_pairs[0][1].reshape(10,1)))
print('Matrix W:\n', W.real)

X_lda = X.dot(W).reshape(-1,1)
assert X_lda.shape == (250,1), "The matrix is not 250x1 dimensional."



################### TYRE 6 ###################

Converting to percentage:

eigenvalue 1: 100.00%
eigenvalue 2: 0.00%
eigenvalue 3: 0.00%
eigenvalue 4: 0.00%
eigenvalue 5: 0.00%
eigenvalue 6: 0.00%
eigenvalue 7: 0.00%
eigenvalue 8: 0.00%
eigenvalue 9: 0.00%
eigenvalue 10: 0.00%
Matrix W:
 [ 0.5329 -0.2942 -0.3371  0.1064  0.2015 -0.0023  0.1073 -0.0832 -0.6577
  0.1136]


In [12]:
final_df=final_df.sort_values(by='Tyre 1',ascending=False)
final_df['Rank1']=range(1,11)
final_df=final_df.sort_values(by='Tyre 2',ascending=False)
final_df['Rank2']=range(1,11)
final_df=final_df.sort_values(by='Tyre 3',ascending=False)
final_df['Rank3']=range(1,11)
final_df=final_df.sort_values(by='Tyre 4',ascending=False)
final_df['Rank4']=range(1,11)
final_df=final_df.sort_values(by='Tyre 5',ascending=False)
final_df['Rank5']=range(1,11)
final_df=final_df.sort_values(by='Tyre 6',ascending=False)
final_df['Rank6']=range(1,11)
final_df.round(3)

,Feature,Tyre 1,Tyre 2,Tyre 3,Tyre 4,Tyre 5,Tyre 6,Rank1,Rank2,Rank3,Rank4,Rank5,Rank6
0,temperature,1.0,1.0,1.0,1.0,1.0,1.0,1,1,1,1,1,1
1,tyre1_press,0.0,0.0,0.0,0.0,0.0,0.0,2,2,2,2,2,2
2,tyre2_press,0.0,0.0,0.0,0.0,0.0,0.0,3,3,3,3,3,3
3,tyre3_press,0.0,0.0,0.0,0.0,0.0,0.0,4,4,4,4,4,4
4,tyre4_press,0.0,0.0,0.0,0.0,0.0,0.0,5,5,5,5,5,5
5,tyre5_press,0.0,0.0,0.0,0.0,0.0,0.0,6,6,6,6,6,6
6,tyre6_press,0.0,0.0,0.0,0.0,0.0,0.0,7,7,7,7,7,7
7,avg_speed,0.0,0.0,0.0,0.0,0.0,0.0,8,8,8,8,8,8
8,time,0.0,0.0,0.0,0.0,0.0,0.0,9,9,9,9,9,9
9,fuel,0.0,0.0,0.0,0.0,0.0,0.0,10,10,10,10,10,10
